# Fine-tuning Llama-3.1 для Natasha

Этот notebook выполняет fine-tuning модели DavidAU/Llama-3.1-DeepSeek-8B-DarkIdol-Instruct-1.2-Uncensored на Google Colab.

## 1. Подготовка окружения

In [ ]:
# Установка зависимостей
!pip install -q torch transformers datasets peft bitsandbytes matplotlib numpy tqdm

In [ ]:
# Подключение Google Drive (если еще не подключен)
from pathlib import Path
if not Path('/content/drive/MyDrive').exists():
    from google.colab import drive
    drive.mount('/content/drive')
    print("✅ Google Drive mounted")
else:
    print("✅ Google Drive already mounted")

In [ ]:
# Проверка GPU
import torch
print(f"GPU: {torch.cuda.get_device_name(0)}")
print(f"VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

## 2. Загрузка датасета

In [ ]:
# Путь к датасету в Google Drive
# Убедись, что файл natasha_finetuning.jsonl находится в /MyDrive/relove_finetuning/
import os
from pathlib import Path

drive_path = Path('/content/drive/MyDrive/relove_finetuning')
drive_path.mkdir(parents=True, exist_ok=True)

dataset_file = drive_path / 'natasha_finetuning.jsonl'
print(f"Dataset path: {dataset_file}")
print(f"Dataset exists: {dataset_file.exists()}")

if dataset_file.exists():
    with open(dataset_file) as f:
        lines = f.readlines()
    print(f"Dataset size: {len(lines)} examples")

## 3. Запуск fine-tuning

In [ ]:
# Импортируем скрипт fine-tuning
import sys
sys.path.insert(0, '/content/drive/MyDrive/relove_finetuning')

# Или копируем скрипт из репозитория
# !wget -q https://raw.githubusercontent.com/YOUR_REPO/scripts/finetuning/finetune_colab.py -O /content/finetune_colab.py

print("✅ Script downloaded")

In [ ]:
# Запускаем fine-tuning
!python /content/drive/MyDrive/relove_finetuning/finetune_colab.py \
  --file /content/drive/MyDrive/relove_finetuning/natasha_finetuning.jsonl \
  --model DavidAU/Llama-3.1-DeepSeek-8B-DarkIdol-Instruct-1.2-Uncensored \
  --epochs 6 \
  --batch-size 1 \
  --lr 1e-4

## 4. Проверка результатов

In [ ]:
# Проверяем сохраненную модель
output_dir = Path('/content/drive/MyDrive/relove_finetuning/model_output')
print(f"Model saved to: {output_dir}")
print(f"Files: {list(output_dir.glob('*'))}")

In [ ]:
# Показываем графики метрик
from PIL import Image
import matplotlib.pyplot as plt

metrics_file = output_dir / 'training_metrics.png'
if metrics_file.exists():
    img = Image.open(metrics_file)
    plt.figure(figsize=(14, 5))
    plt.imshow(img)
    plt.axis('off')
    plt.show()
else:
    print("Metrics file not found")

In [ ]:
# Показываем статистику обучения
import json

stats_file = output_dir / 'training_stats.json'
if stats_file.exists():
    with open(stats_file) as f:
        stats = json.load(f)
    print(json.dumps(stats, indent=2))
else:
    print("Stats file not found")